In [46]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

import sys

sys.path.insert(0, '../visualize/')
sys.path.insert(0, '../models/')

from tcors_functions import *
from counterprop import *
from visualize import *

random_state = 42


In [12]:
df = load_features('model_features.pkl')
df.head()

,screen_id,week,project,site,dose,screen_sex,screen_age,carmine_nicotine,prp_change,prp_change_bin,prp_change_bin_label
37,J-A003,week12,project 1,uvm,15.8,Female,32,16.7,0.487500,"(0.4, 0.5]",14
50,J-A004,week12,project 1,uvm,0.4,Female,38,15.9,0.725225,"(0.7, 0.8]",17
76,J-A007,week12,project 1,uvm,15.8,Female,44,16.2,0.971223,"(0.9, 1.0]",19
89,J-A010,week12,project 1,uvm,15.8,Female,28,15.8,0.895522,"(0.8, 0.9]",18
117,J-A015,week12,project 1,uvm,15.8,Female,25,16.9,1.315152,"(1.3, 1.4]",23


Preprocessing

In [32]:
cat_targets = ['prp_change_bin_label']
num_targets = ['prp_change']
cat_predictors = ['project', 'site', 'dose', 'screen_sex']
num_predictors = ['screen_age', 'carmine_nicotine']

X = df[cat_predictors + num_predictors]
y = df[cat_targets + num_targets]


,prp_change_bin_label,prp_change
37,14,0.487500
50,17,0.725225
76,19,0.971223
89,18,0.895522
117,23,1.315152
...,...,...
8986,19,0.968750
8999,21,1.140845
9017,17,0.715026
9030,24,1.454545


* Scale data using mean from training data to prevent leakage
* One-hot encode categorical data

In [49]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import set_config
from collections import namedtuple

def fit_linear(num_predictors: list, cat_predictors: list, random_state=42):

    scaler = StandardScaler()
    encoder = OneHotEncoder(sparse=False)


    preprocessor = ColumnTransformer(
        transformers = [
            ("num", scaler, num_predictors),
            ("cat", encoder, cat_predictors)
        ]
    )

    reg = Pipeline(
        steps = [("preprocessor", preprocessor), ("regression", LinearRegression())]
    )

    X_train, X_test, y_train, y_test = train_test_split(X, y['prp_change'], test_size=0.2, random_state=random_state)

    # estimate training performance
    cv_scores = cross_val_score(reg, X_train, y_train, cv = 5)

    # assess test set performance fit to all training data
    reg.fit(X_train, y_train)
    test_score = reg.score(X_test, y_test)

    summary = namedtuple("summary", ["cv_scores", "test_score"])

    return summary(cv_scores, test_score)




0.13870750193507075

In [38]:
set_config(display="diagram")
reg

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['screen_age',
                                                   'carmine_nicotine']),
                                                 ('cat',
                                                  OneHotEncoder(sparse=False),
                                                  ['project', 'site', 'dose',
                                                   'screen_sex'])])),
                ('regression', LinearRegression())])

In [15]:
y_train

,prp_change,prp_change_bin,prp_change_bin_label
5276,0.085714,"(-2.22e-16, 0.1]",10
8616,0.346939,"(0.3, 0.4]",13
7858,3.170213,"(2.0, inf]",30
3027,1.000000,"(1.0, 1.1]",20
561,0.140000,"(0.1, 0.2]",11
...,...,...,...
2200,0.586207,"(0.5, 0.6]",15
5165,0.652273,"(0.6, 0.7]",16
6455,1.065217,"(1.0, 1.1]",20
7883,0.436137,"(0.4, 0.5]",14


## Linear Model
### Regression


In [5]:
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import make_pipeline


pipe = make_pipeline(OneHotEncoder(), StandardScaler(), LinearRegression())
pipe.fit(X_train, y_train)

Pipeline(steps = [('standardscaler', StandardScaler()),
                    ('linearregression', LinearRegression())])
pipe.score(X_test, y_test)

ValueError: could not convert string to float: 'J-C032'